# TITANIC MODEL TRAINING

In [51]:
# importing all the essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [52]:
# Loading the dataset from seaborn library of titanic dataset
df = sns.load_dataset('titanic')

## Taking a bird eye view of data

In [53]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## PERFORMING EDA

In [54]:
# Checking the datatypes of columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [55]:
# Checking is there is any null values
df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

- Deck column has a lot of missing values so we drop it
- Embarked and Embarked_town are same columns so we drop one and fill the missing values using mode
- Age has a good number of missing values so we alos fill it with medain
- Columns which are not correlated to survived we drop it from the data to gain more effiecncy from model

In [56]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [57]:
# Removing all the unnecessary columns
df.drop(columns=['embark_town' , 'deck' , 'alive', 'alone' , 'adult_male' , 'who'] , axis=1 , inplace=True)

In [58]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class
0,0,3,male,22.0,1,0,7.2500,S,Third
1,1,1,female,38.0,1,0,71.2833,C,First
2,1,3,female,26.0,0,0,7.9250,S,Third
3,1,1,female,35.0,1,0,53.1000,S,First
4,0,3,male,35.0,0,0,8.0500,S,Third


In [59]:
df.sample(5)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class
220,1,3,male,16.0,0,0,8.0500,S,Third
770,0,3,male,24.0,0,0,9.5000,S,Third
517,0,3,male,NaN,0,0,24.1500,Q,Third
709,1,3,male,NaN,1,1,15.2458,C,Third
151,1,1,female,22.0,1,0,66.6000,S,First


In [60]:
# Mapping all values with 0 and 1
df['sex'] =  df['sex'].map({'male':0 , 'female':1})
df['embarked'] = df['embarked'].map({'S':0,'Q':1, 'C':2})
df['class'] = df['class'].map({'First':1 , 'Second':2 , 'Third':3})

In [61]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class
0,0,3,0,22.0,1,0,7.2500,0.0,3
1,1,1,1,38.0,1,0,71.2833,2.0,1
2,1,3,1,26.0,0,0,7.9250,0.0,3
3,1,1,1,35.0,1,0,53.1000,0.0,1
4,0,3,0,35.0,0,0,8.0500,0.0,3


In [62]:
df.isnull().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
class         0
dtype: int64

In [63]:
df['embarked'].fillna(df['embarked'].mode()[0], inplace=True)

C:\Users\Home\AppData\Local\Temp\ipykernel_6532\1964997694.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['embarked'].fillna(df['embarked'].mode()[0], inplace=True)


In [64]:
df['age'].fillna(df['age'].median(), inplace=True)

C:\Users\Home\AppData\Local\Temp\ipykernel_6532\1654433547.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['age'].fillna(df['age'].median(), inplace=True)


In [65]:
df.isnull().sum()

survived    0
pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
embarked    0
class       0
dtype: int64

## GOING TO TRAIN MODEL

In [66]:
lr = LogisticRegression()


In [67]:
X = df.drop(['survived'], axis=1)
y = df['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [68]:
lr.fit(X_train , y_train)

C:\Users\Home\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [69]:
accuracy = lr.score(X_test, y_test)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.81


In [70]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [71]:
# KNN model
knn = KNeighborsClassifier(n_neighbors=3) 

# Decision Tree model
dt = DecisionTreeClassifier(criterion='gini', max_depth=5)  


In [72]:

dt.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=5)

In [73]:
accuracy = dt.score(X_test, y_test)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.80


In [74]:
knn.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=3)

In [75]:
accuracy = knn.score(X_test, y_test)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.73


In [76]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Features & Target
X = df.drop('survived', axis=1)
y = df['survived']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistic Regression
lr = LogisticRegression(max_iter=200)
lr.fit(X_train, y_train)
pred_lr = lr.predict(X_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, pred_lr))

# Decision Tree
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
pred_dt = dt.predict(X_test)
print("Decision Tree Accuracy:", accuracy_score(y_test, pred_dt))

# KNN
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
pred_knn = knn.predict(X_test)
print("KNN Accuracy:", accuracy_score(y_test, pred_knn))


Logistic Regression Accuracy: 0.8100558659217877
Decision Tree Accuracy: 0.776536312849162
KNN Accuracy: 0.7094972067039106
